### The following code converts GTFS files
## For GTFS files with calendar.txt file

In [ ]:
import pandas as pd

# Read CSV files
calendar = pd.read_csv('calendar.txt')
trips = pd.read_csv('trips.txt')
stop_times = pd.read_csv('stop_times.txt')
stops = pd.read_csv('stops.txt')
routes = pd.read_csv('routes.txt')

# Merge dataframes
merged = trips.merge(calendar, on='service_id').merge(stop_times, on='trip_id').merge(stops, on='stop_id').merge(routes, on='route_id')

calendar_service_id = set(calendar['service_id'])

# Calculate the number of stops in a normal week for each stop
merged['week_stops'] = merged['monday'] + merged['tuesday'] + merged['wednesday'] + merged['thursday'] + merged['friday'] + merged['saturday'] + merged['sunday']

print(merged['week_stops'].unique())

# Group by route_type and stop_id, summing the week_stops
grouped = merged.groupby(['route_type', 'stop_id']).agg({'week_stops': 'sum', 'stop_lat': 'first', 'stop_lon': 'first', 'stop_name': 'first'}).reset_index()

# Write the results to separate output files for each route_type
for route_type in grouped['route_type'].unique():
    output = grouped[grouped['route_type'] == route_type]
    output.to_csv(f'output_route_type_{route_type}.csv', index=False)


## For GTFS files with calendar_dates.txt file

In [ ]:
from datetime import datetime

# Read CSV files
trips = pd.read_csv("trips.txt")
stop_times = pd.read_csv("stop_times.txt")
stops = pd.read_csv("stops.txt")
routes = pd.read_csv("routes.txt")
calendar_dates = pd.read_csv("calendar_dates.txt")

# Convert the 'date' column to a pandas datetime object
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

# Create a new column with the day of the week in text format
calendar_dates['day'] = calendar_dates['date'].dt.day_name()

service_count = calendar_dates.groupby('service_id')['day'].nunique().reset_index()

# Merge DataFrames
merged = trips.merge(stop_times, on='trip_id')
merged = merged.merge(routes, on='route_id')
merged = merged.merge(stops, on='stop_id')
merged = merged.merge(service_count, on='service_id')

for route_type in merged['route_type'].unique():
    # Filter only route type of interest
    result = merged.loc[merged['route_type'] == route_type]
    
    # Group by stop_id and sum the day value
    grouped = result.groupby('stop_id').agg({'day': 'sum', 'stop_lat': 'first', 'stop_lon': 'first', 'stop_name': 'first', 'stop_id': 'first', 'route_type': 'first'})
    
    #Write to CSV
    grouped.to_csv(f"route_type_{route_type}_dates.csv", index = False)
